# Module 5 - Batch - Spark - HOMEWORK

Create the session

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/08 17:27:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/08 17:27:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Find out the version

In [3]:
spark.version


'3.3.2'

Read the parquet data

In [4]:
df_yellow = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [5]:
df_yellow.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 02:30:44|  2024-10-01 02:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [6]:
df_yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

Repartition the Dataframe to 4 partitions and save it to parquet.

In [7]:
output_path = 'data/pq/yellow/2024/10/'

In [ ]:
df_yellow \
    .repartition(4) \
    .write.parquet(output_path)

How many taxi trips were there on the 15th of October?

In [9]:
from pyspark.sql import functions as F

In [10]:
df_yellow.createOrReplaceTempView('trips_data')

In [11]:
df_result = spark.sql("""
    SELECT 
        COUNT(*) AS number_trips
    FROM 
        trips_data
    WHERE
        tpep_pickup_datetime BETWEEN '2024-10-15 00:00:00' AND '2024-10-15 23:59:59'
    """).show()

+------------+
|number_trips|
+------------+
|      125567|
+------------+



What is the length of the longest trip in the dataset in hours?

In [12]:
df_yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [23]:
df_diff = spark.sql("""
    SELECT 
        (UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / 3600 AS hours_dif
    FROM 
        trips_data
    ORDER BY 
        hours_dif DESC
    LIMIT 1;
""").show()

+------------------+
|         hours_dif|
+------------------+
|162.61777777777777|
+------------------+



Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?

In [27]:
df_zone = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

In [28]:
df_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [29]:
df_zone.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



Changes the LocationID type from string to integer

In [31]:
from pyspark.sql.types import IntegerType, TimestampType

In [32]:
df_zone = df_zone.withColumn("LocationID", df_zone["LocationID"].cast(IntegerType()))

In [33]:
df_zone.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [37]:
df_zone.createOrReplaceTempView('zones')

In [39]:
df_last_zone = spark.sql("""
    SELECT 
        COUNT(trips_data.tpep_pickup_datetime) as number_picksups
        ,z_pu.Zone as pickup_location_zone
    FROM
        trips_data
        INNER JOIN zones as z_pu ON z_pu.LocationID = trips_data.PULocationID
    GROUP BY
        z_pu.Zone
    ORDER BY
        number_picksups ASC
    LIMIT 1;
""").show()

+---------------+--------------------+
|number_picksups|pickup_location_zone|
+---------------+--------------------+
|              1|Governor's Island...|
+---------------+--------------------+

